IMPORTAÇÃO DE BIBLIOTECAS

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import sklearn 

CARREGAMENTO DE ARQUIVOS EM DATAFRAMES NO PANDAS

> TABELAS DIMENSÃO E TABELA FATO



In [ ]:
# Importando as dimensões e fato

dim_circuits = pd.read_csv('DIM_CIRCUITS.CSV', encoding = "ISO-8859-1")
dim_constructors = pd.read_csv('DIM_CONSTRUCTORS.CSV', encoding = "ISO-8859-1")
dim_drivers = pd.read_csv('DIM_DRIVERS.CSV', encoding = "ISO-8859-1")
dim_races = pd.read_csv('DIM_RACES.CSV')
fact_results = pd.read_csv('FACT_RESULTS.csv')

In [ ]:
#dim_races

JOINING TABLES

In [ ]:
# CRIANDO NOVA COLUNA CONCATENANDO O NOME DO PILOTO
dim_drivers['DRIVER_NAME'] = dim_drivers['DRIVER_FORENAME'] + ' ' + dim_drivers['DRIVER_SURNAME']

# CONVERTENDO RACE_DATE DE OBJECT PARA DATETIME
fact_results['RACE_DATE'] = pd.to_datetime(fact_results['RACE_DATE'])

In [ ]:
df_fact = fact_results.merge(dim_races, how = 'inner', on = 'RACE_ID',suffixes=[None, None])
df_fact = df_fact.merge(dim_circuits, how = 'inner', on = 'CIRCUIT_ID', suffixes =[None, None])
df_fact = df_fact.merge(dim_drivers, how = 'inner', on = 'DRIVER_ID', suffixes =[None, None])
df_fact = df_fact.merge(dim_constructors, how = 'inner', on = 'CONSTRUCTOR_ID', suffixes =[None, None])

In [ ]:
#df_fact.dtypes

In [ ]:
#df_copy.groupby('DRIVER_NAME','RACE_YEAR')['DRIVER_POSITION_ORDER'].min()

CONVERSÃO DE DADOS E CRIAÇÃO DE NOVAS COLUNAS

In [ ]:
#CRIANDO NOVA COLUNA SE O PILOTO COMEÇOU EM PRIMEIRO E LIDEROU DURANTE TODA A PROVA

laps = pd.read_csv('lap_times.csv')

laps_max = laps.groupby(['raceId','driverId']).max(['position'])
laps_min = laps.groupby(['raceId','driverId']).min(['position'])
laps_max['WIN_EVERY_LAP'] = np.where((laps_max['position'] == 1) & (laps_min['position'] == 1), True, False)
win_every_lap = laps_max.drop(['lap','position','milliseconds'], axis=1)

win_every_lap

WIN_EVERY_LAP
raceId driverId               
1      1                 False
       2                 False
       3                 False
       4                 False
       6                 False
...                        ...
1055   847               False
       849               False
       852               False
       853               False
       854               False

[9368 rows x 1 columns]

In [ ]:
# JOIN DA TABELA DE VOLTAS COM COLUNA DE LIDERANÇA DE PROVA E TABELA FATO

df_fact = df_fact.merge(win_every_lap, how = 'left', left_on = ['RACE_ID','DRIVER_ID'], right_on = ['raceId','driverId'], suffixes = [None, None])

In [ ]:
# QUANTIDADE DE LINHAS NULAS POR COLUNA
#df_fact.isna().sum()
#pd.unique(fact_results['RACE_DRIVER_POINTS'])
# CONSTRUCTOR_RESULTS_STATUS = DROP?
# RACE TIME = DROP?
# DRIVER POSITION (FINISH OU ORDER) = DROP?

In [ ]:
print(df_fact[df_fact['RACE_YEAR'] >= 2019]['FASTEST_LAP_TIME'].shape[0])
print(df_fact[df_fact['RACE_YEAR'] >= 2019]['FASTEST_LAP_TIME'].dropna().shape[0])
print(df_fact[df_fact['RACE_YEAR'] >= 2019]['FASTEST_LAP_TIME'].shape[0]-df_fact[df_fact['RACE_YEAR'] >= 2019]['FASTEST_LAP_TIME'].dropna().shape[0])


840
821
19


In [ ]:
df_copy = df_fact.copy()
df_copy.drop(['DRIVER_NUMBER','DRIVER_CODE','FASTEST_LAP_NUMBER','RACE_FINISH_TIME','DRIVER_FINISH_POSITION','RACE_TIME'],axis=1,inplace=True)
df_copy.isna().sum()


# CONSTRUCTOR_RESULTS_POINTS        NA = 0 
# DRIVER_STANDINGS_POINTS           NA = 0
# CONSTRUCTOR_RESULTS_STATUS        NA = 'NÃO INFORMADO'
# DRIVER_STANDINGS_WINS             NA = 0
# CONSTRUCTOR_STANDINGS_POINTS      NA = 0
# CONSTRUCTOR_STANDINGS_WINS        NA = 0
# DRIVER_FINISH_POSITION            NA = 0
# CONSTRUCTOR_STANDINGS_POSITION    NA = 0
# DRIVER_STANDINGS_POSITION         NA = 0
# FASTEST_LAP_RANK                  NA = 0
# WIN_EVERY_LAP                     NA = FALSE                  

# DRIVER_NUMBER                     DROP
# DRIVER_CODE                       DROP
# FASTEST_LAP_NUMBER                
# RACE_FINISH_TIME                  
# DRIVER_FINISH_POSITION            
# RACE_ROUND                        
# RACE_TIME                         

#I've decided to impute the median of each metric instead of missing values
#from sklearn.impute import SimpleImputer
#imputer = SimpleImputer(missing_values=np.nan, strategy='median')
#imputer = imputer.fit(shoppers[['found_rate', 'accepted_rate', 'rating']])

#shoppers[['found_rate', 'accepted_rate', 'rating']] = imputer.transform(shoppers[['found_rate', 'accepted_rate', 'rating']].values)


RESULT_ID                             0
DRIVER_ID                             0
CONSTRUCTOR_ID                        0
RACE_ID                               0
CIRCUIT_ID                            0
CONSTRUCTOR_RESULTS_POINTS         1573
CONSTRUCTOR_RESULTS_STATUS        25006
CONSTRUCTOR_STANDINGS_POINTS       1875
CONSTRUCTOR_STANDINGS_POSITION     1875
CONSTRUCTOR_STANDINGS_WINS         1875
DRIVER_STANDINGS_POINTS             481
DRIVER_STANDINGS_POSITION           481
DRIVER_STANDINGS_WINS               481
RACE_YEAR                             0
RACE_ROUND                            0
RACE_DATE                             0
GRID_POSITION                         0
DRIVER_POSITION_ORDER                 0
RACE_DRIVER_POINTS                    0
COMPLETED_LAPS                        0
RACE_FINISH_TIME_MILLISECONDS     18407
FASTEST_LAP_RANK                  18249
FASTEST_LAP_TIME                  18413
FASTEST_LAP_SPEED                 18413
STATUS                                0


In [ ]:
df_copy['CIRCUIT_NAME'].unique()

#Suzuka Circuit
#Circuit de Barcelona-Catalunya
#Baku City Circuit

array(['Albert Park Grand Prix Circuit', 'Autódromo José Carlos Pace',
       'Autodromo Enzo e Dino Ferrari', 'Silverstone Circuit',
       'Circuit de Barcelona-Catalunya', 'Nürburgring',
       'Circuit de Monaco', 'Circuit Gilles Villeneuve',
       'Circuit de Nevers Magny-Cours', 'A1-Ring', 'Hockenheimring',
       'Hungaroring', 'Circuit de Spa-Francorchamps',
       'Autodromo Nazionale di Monza', 'Sepang International Circuit',
       'Indianapolis Motor Speedway', 'Suzuka Circuit',
       'Autódromo Juan y Oscar Gálvez', 'Autódromo do Estoril',
       'Circuito de Jerez', 'Okayama International Circuit',
       'Adelaide Street Circuit', 'Autódromo Hermanos Rodríguez',
       'Phoenix street circuit', 'Autódromo Internacional Nelson Piquet',
       'Kyalami', 'Donington Park', 'Circuit Paul Ricard',
       'Detroit Street Circuit', 'Brands Hatch',
       'Bahrain International Circuit', 'Shanghai International Circuit',
       'Istanbul Park', 'Fuji Speedway', 'Valencia Stree

In [ ]:
# SÓ CONSEGUIMOS SABER A VELOCIDADE DAS VOLTAS A PARTIR DE 2004

# VELOCIDADE MÉDIA AO LONGO DOS ÚLTIMOS 17 ANOS
year_speed_mean = df_copy.groupby(['RACE_YEAR'])['FASTEST_LAP_SPEED'].mean().dropna().reset_index()

# VELOCIDADE MÉDIA NAS PISTAS DE MÔNACO, SUZUKA, BARCELONA E BAKU NOS ÚLTIMOS 17 ANOS
year_speed_mean_monaco = df_copy.groupby(['RACE_YEAR', 'CIRCUIT_NAME'])['FASTEST_LAP_SPEED'].mean().dropna().reset_index()
year_speed_mean_monaco = year_speed_mean_monaco[year_speed_mean_monaco['CIRCUIT_NAME'] == 'Circuit de Monaco']
year_speed_mean_suzuka = df_copy.groupby(['RACE_YEAR', 'CIRCUIT_NAME'])['FASTEST_LAP_SPEED'].mean().dropna().reset_index()
year_speed_mean_suzuka = year_speed_mean_suzuka[year_speed_mean_suzuka['CIRCUIT_NAME'] == 'Suzuka Circuit']
year_speed_mean_barcelona = df_copy.groupby(['RACE_YEAR', 'CIRCUIT_NAME'])['FASTEST_LAP_SPEED'].mean().dropna().reset_index()
year_speed_mean_barcelona = year_speed_mean_barcelona[year_speed_mean_barcelona['CIRCUIT_NAME'] == 'Circuit de Barcelona-Catalunya']
year_speed_mean_baku = df_copy.groupby(['RACE_YEAR', 'CIRCUIT_NAME'])['FASTEST_LAP_SPEED'].mean().dropna().reset_index()
year_speed_mean_baku = year_speed_mean_baku[year_speed_mean_baku['CIRCUIT_NAME'] == 'Baku City Circuit']



#year_speed_mean_monaco                  # THE MOST DIFFICULT EVENT IN F1
#year_speed_mean_suzuka                 # ONE OF THE MOST DIFFICULT
#year_speed_mean_barcelona              # WELL ROUNDED
#year_speed_mean_baku                   # MAIS RÁPIDO

#CRIAÇÃO DE VARIÁVEIS CONTENDO INSIGHTS

In [ ]:
df_fact.groupby('DRIVER_NAME')['DRIVER_NATIONALITY'].unique()

DRIVER_NAME
Adolf Brudes            [German]
Adolfo Cruz          [Argentine]
Adrian Sutil            [German]
Adrián Campos          [Spanish]
Aguri Suzuki          [Japanese]
                        ...     
Zsolt Baumgartner    [Hungarian]
Élie Bayol              [French]
Éric Bernard            [French]
Érik Comas              [French]
Óscar González       [Uruguayan]
Name: DRIVER_NATIONALITY, Length: 853, dtype: object

In [ ]:
driver_points_sum(2019)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:4: FutureWarning: Indexing with multiple keys (implicitly converted to a tuple of keys) will be deprecated, use a list instead.
  after removing the cwd from sys.path.


,RACE_YEAR,DRIVER_NAME,RACE_DRIVER_POINTS
11,2019,Lewis Hamilton,413.0
19,2019,Valtteri Bottas,326.0
12,2019,Max Verstappen,278.0
3,2019,Charles Leclerc,264.0
17,2019,Sebastian Vettel,240.0
2,2019,Carlos Sainz,96.0
14,2019,Pierre Gasly,95.0
0,2019,Alexander Albon,92.0
4,2019,Daniel Ricciardo,54.0
18,2019,Sergio Pérez,52.0


In [ ]:
# PILOTOS COM MAIS VITÓRIAS NO STANDINGS POR ANO

def driver_points_sum(year):
  driver_points_sum = df_fact[df_fact['RACE_YEAR'] == year].groupby(['RACE_YEAR','DRIVER_NAME'])['RACE_DRIVER_POINTS','DRIVER_NATIONALITY'].sum().reset_index()
  driver_points_sum = driver_points_sum.sort_values(by = ['RACE_DRIVER_POINTS'], ascending=False)
  return driver_points_sum

# CONSTRUTORAS COM MAIS VITÓRIAS NO STANDINGS POR ANO
def constructor_wins(year):
  constructor_wins = df_fact[df_fact['RACE_YEAR'] == year].groupby(['RACE_YEAR','CONSTRUCTOR_NAME'])['CONSTRUCTOR_STANDINGS_POINTS'].max().reset_index()
  constructor_wins = constructor_wins.sort_values(by = ['CONSTRUCTOR_STANDINGS_POINTS'], ascending=False)
  return constructor_wins

# PILOTOS COM MAIS POLE POSITIONS
def driver_pole_count(year=''):
  if year == '':
    driver_pole = df_fact[df_fact['GRID_POSITION'] == 1]
    driver_pole = driver_pole.groupby('DRIVER_NAME')['GRID_POSITION'].count().reset_index()
    driver_pole = driver_pole.sort_values(by = 'GRID_POSITION', ascending=False)
  else:
    driver_pole = df_fact[(df_fact['RACE_YEAR'] == year) & (df_fact['GRID_POSITION'] == 1)]
    driver_pole = driver_pole.groupby('DRIVER_NAME')['GRID_POSITION'].count().reset_index()
    driver_pole = driver_pole.sort_values(by = 'GRID_POSITION', ascending=False)
  return driver_pole

# PILOTOS COM MAIS PÓDIOS
def driver_podium_count(year=''):
  if year == '': 
    drivers_podium = df_fact[df_fact['DRIVER_POSITION_ORDER'] <= 3]
    drivers_podium = drivers_podium.groupby('DRIVER_NAME')['DRIVER_POSITION_ORDER'].count().reset_index()
    drivers_podium = drivers_podium.sort_values(by = 'DRIVER_POSITION_ORDER', ascending=False)
  else:
    drivers_podium = df_fact[(df_fact['DRIVER_POSITION_ORDER'] <= 3) & (df_fact['RACE_YEAR'] == year)]
    drivers_podium = drivers_podium.groupby('DRIVER_NAME')['DRIVER_POSITION_ORDER'].count().reset_index()
    drivers_podium = drivers_podium.sort_values(by = 'DRIVER_POSITION_ORDER', ascending=False)
  return drivers_podium

# PILOTOS COM MAIS VITÓRIAS EM CORRIDAS

def driver_win_count(year=''):
  if year == '': 
    driver_win_count = df_fact[df_fact['DRIVER_POSITION_ORDER'] == 1]
    driver_win_count = driver_win_count.groupby('DRIVER_NAME')['DRIVER_POSITION_ORDER'].count().reset_index()
    driver_win_count = driver_win_count.sort_values(by = 'DRIVER_POSITION_ORDER', ascending=False)
  else:
    driver_win_count = df_fact[(df_fact['DRIVER_POSITION_ORDER'] == 1) & (df_fact['RACE_YEAR'] == year)]
    driver_win_count = driver_win_count.groupby('DRIVER_NAME')['DRIVER_POSITION_ORDER'].count().reset_index()
    driver_win_count = driver_win_count.sort_values(by = 'DRIVER_POSITION_ORDER', ascending=False)
  return driver_win_count

# CLASSIFICAÇÃO DO CAMPEONATO DE PILOTOS

def driver_standings(year):
  points = driver_points_sum(year)
  pole = driver_pole_count(year)
  podium = driver_podium_count(year)
  win = driver_win_count(year)
  stand = points.merge(pole, how = 'outer', on = 'DRIVER_NAME').merge(win, how = 'outer', on = 'DRIVER_NAME').merge(podium, how = 'outer', on = 'DRIVER_NAME')
  stand = stand.rename(columns={
      "RACE_YEAR": "SEASON",
      "RACE_DRIVER_POINTS": "POINTS",
      "GRID_POSITION": "POLE_POSITIONS",
      "DRIVER_POSITION_ORDER_x": "WINS",
      "DRIVER_POSITION_ORDER_y": "PODIUMS"})
  return stand

#driver_win_count(2020)
driver_podium_count(2021)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:4: FutureWarning: Indexing with multiple keys (implicitly converted to a tuple of keys) will be deprecated, use a list instead.
  after removing the cwd from sys.path.


,SEASON,DRIVER_NAME,POINTS,POLE_POSITIONS,WINS,PODIUMS
0,2021,Lewis Hamilton,94.0,2.0,3.0,4.0
1,2021,Max Verstappen,80.0,1.0,1.0,4.0
2,2021,Valtteri Bottas,47.0,1.0,NaN,3.0
3,2021,Lando Norris,41.0,NaN,NaN,1.0
4,2021,Charles Leclerc,40.0,NaN,NaN,NaN
5,2021,Sergio Pérez,32.0,NaN,NaN,NaN
6,2021,Daniel Ricciardo,24.0,NaN,NaN,NaN
7,2021,Carlos Sainz,20.0,NaN,NaN,NaN
8,2021,Esteban Ocon,10.0,NaN,NaN,NaN
9,2021,Pierre Gasly,8.0,NaN,NaN,NaN


In [ ]:
driver_wins = df_fact[df_fact['RACE_YEAR'] == 2020].groupby(['RACE_YEAR','DRIVER_NAME'])['RACE_DRIVER_POINTS'].sum().reset_index()
driver_wins = driver_wins.sort_values(by = ['RACE_DRIVER_POINTS'], ascending=False)

driver_wins

,RACE_YEAR,DRIVER_NAME,RACE_DRIVER_POINTS
13,2020,Lewis Hamilton,347.0
22,2020,Valtteri Bottas,223.0
14,2020,Max Verstappen,214.0
21,2020,Sergio Pérez,125.0
4,2020,Daniel Ricciardo,119.0
0,2020,Alexander Albon,105.0
2,2020,Carlos Sainz,105.0
3,2020,Charles Leclerc,98.0
12,2020,Lando Norris,97.0
17,2020,Pierre Gasly,75.0


In [ ]:
    driver_pole = df_fact[(df_fact['RACE_YEAR'] == 2021) & (df_fact['GRID_POSITION'] == 1)]
    driver_pole = driver_pole.groupby('DRIVER_NAME')['GRID_POSITION'].count().reset_index()
    driver_pole = driver_pole.sort_values(by = 'GRID_POSITION', ascending=False)
    driver_pole

,DRIVER_NAME,GRID_POSITION
0,Lewis Hamilton,2
1,Max Verstappen,1
2,Valtteri Bottas,1


In [ ]:
# PILOTOS COM MAIS PÓDIOS

drivers_podium = df_fact[(df_fact['DRIVER_POSITION_ORDER'] <= 3) & (df_fact['DRIVER_POSITION_ORDER'] > 0)]
drivers_podium = drivers_podium.groupby('DRIVER_NAME')['DRIVER_POSITION_ORDER'].count().reset_index()
drivers_podium = drivers_podium.sort_values(by = 'DRIVER_POSITION_ORDER', ascending=False)

#drivers_podium

In [ ]:
# PILOTOS COM MAIS VITÓRIAS EM CORRIDAS

drivers_race_win = df_fact[df_fact['DRIVER_POSITION_ORDER'] == 1]
drivers_race_win = drivers_race_win.groupby('DRIVER_NAME')['DRIVER_POSITION_ORDER'].count().reset_index()
drivers_race_win = drivers_race_win.sort_values(by = 'DRIVER_POSITION_ORDER', ascending=False)

#drivers_race_win

In [ ]:
# PILOTO COM MAIS VOLTAS RAPIDAS

race_best_lap = df_fact[df_fact['FASTEST_LAP_RANK'] == 1].groupby('DRIVER_NAME').sum().reset_index()
race_best_lap = race_best_lap.sort_values(by = 'FASTEST_LAP_RANK', ascending=False)
l = ['DRIVER_NAME','FASTEST_LAP_RANK', 'FASTEST_LAP_NUMBER', 'FASTEST_LAP_SPEED', 'FASTEST_LAP_TIME']
race_best_lap = race_best_lap.reindex(l,axis=1)
race_best_lap = race_best_lap[l].head(10)
#race_best_lap

In [ ]:
# AS VOLTAS NO DATASET SÓ FORAM REGISTRADAS A PARTIR DE 1996
# df_fact[df_fact.WIN_EVERY_LAP == True].groupby('RACE_YEAR').count()['WIN_EVERY_LAP']

In [ ]:
# GRAND CHELEM ou GRAND SLAM
#
# GRID_POSITION == 1                                       # POLE POSITION
# FASTEST_LAP_RANK == 1                                    # VOLTA MAIS RÁPIDA
# WIN_EVERY_LAP == True                                    # LIDEROU DE PONTA A PONTA

grand_slam = df_copy.query('GRID_POSITION == 1 and FASTEST_LAP_RANK == 1 and WIN_EVERY_LAP == True')[['DRIVER_NAME','RACE_NAME', 'RACE_YEAR','FASTEST_LAP_TIME','FASTEST_LAP_SPEED']]
grand_slam_count = grand_slam.groupby('DRIVER_NAME').count().reset_index()

#grand_slam

In [ ]:
# VENCEDORES DO GP DE MONACO (MAIS PRESTIGIADO DA F1)

# df_fact['CIRCUIT_NAME'].unique()                       # VERIFICAR O NOME DO CIRCUITO

monaco_driver_wins = df_fact.query("CIRCUIT_NAME == 'Circuit de Monaco' and DRIVER_POSITION_ORDER == 1")[['DRIVER_NAME','RACE_YEAR']]
monaco_driver_wins = monaco_driver_wins.groupby('DRIVER_NAME').count().sort_values(by='RACE_YEAR',ascending=False).reset_index()

monaco_driver_wins

,DRIVER_NAME,RACE_YEAR
0,Ayrton Senna,6
1,Graham Hill,5
2,Michael Schumacher,5
3,Alain Prost,4
4,Nico Rosberg,3
5,Lewis Hamilton,3
6,Jackie Stewart,3
7,Stirling Moss,3
8,Juan Fangio,2
9,Maurice Trintignant,2


In [ ]:
# EQUIPE QUE MAIS VENCEU EM MONACO

monaco_constructor_wins = df_fact.query("CIRCUIT_NAME == 'Circuit de Monaco' and DRIVER_POSITION_ORDER == 1")[['CONSTRUCTOR_NAME','RACE_YEAR']]
monaco_constructor_wins = monaco_constructor_wins.groupby('CONSTRUCTOR_NAME').count().sort_values(by='RACE_YEAR',ascending=False).reset_index()

monaco_constructor_wins

,CONSTRUCTOR_NAME,RACE_YEAR
0,McLaren,15
1,Ferrari,9
2,Mercedes,5
3,BRM,5
4,Team Lotus,4
5,Red Bull,4
6,Williams,3
7,Tyrrell,3
8,Benetton,2
9,Renault,2


In [1]:
# PERGUNTA: QUAL O PERCENTUAL DE PILOTOS QUE LARGAM NA POLE E GANHAM PROVAS??
#           HÁ CIRCUITOS ONDE É MAIS DIFÍCIL GANHAR SEM TER LARGADO NA POLE?


#driver_wins_vs_grid = df_fact.query("DRIVER_POSITION_ORDER == 1 and GRID_POSITION == 1 and RACE_DATE >= 2004")[['DRIVER_NAME','CIRCUIT_NAME','RACE_YEAR','DRIVER_POSITION_ORDER','GRID_POSITION']]
#driver_wins_vs_grid = driver_wins_vs_grid.groupby(['CIRCUIT_NAME']).count().sort_values(by='RACE_YEAR',ascending=False)

driver_wins_vs_grid_lead = df_fact.query("DRIVER_POSITION_ORDER == 1 and GRID_POSITION == 1 and WIN_EVERY_LAP == True")[[GRID_POSITION']]
driver_wins_vs_grid_lead = driver_wins_vs_grid_lead[driver_wins_vs_grid_lead['WIN_EVERY_LAP'] == True].groupby(['CIRCUIT_NAME']).count().sort_values(by='RACE_YEAR',ascending=False)

driver_wins_vs_grid_lead

#monaco_driver_wins_vs_grid = df_fact.query("CIRCUIT_NAME == 'Circuit de Monaco' and DRIVER_POSITION_ORDER == 1")[['DRIVER_NAME','RACE_YEAR','DRIVER_POSITION_ORDER','GRID_POSITION']]
#barcelona_driver_wins_vs_grid = df_fact.query("CIRCUIT_NAME == 'Circuit de Barcelona-Catalunya' and DRIVER_POSITION_ORDER == 1")[['DRIVER_NAME','RACE_YEAR','DRIVER_POSITION_ORDER','GRID_POSITION']]
#suzuka_driver_wins_vs_grid = df_fact.query("CIRCUIT_NAME == 'Suzuka Circuit' and DRIVER_POSITION_ORDER == 1")[['DRIVER_NAME','RACE_YEAR','DRIVER_POSITION_ORDER','GRID_POSITION']]
#interlagos_driver_wins_vs_grid = df_fact.query("CIRCUIT_NAME == 'Autódromo José Carlos Pace' and DRIVER_POSITION_ORDER == 1")[['DRIVER_NAME','RACE_YEAR','DRIVER_POSITION_ORDER','GRID_POSITION']]

#monaco_driver_wins_vs_grid
#barcelona_driver_wins_vs_grid
#suzuka_driver_wins_vs_grid
#interlagos_driver_wins_vs_grid

SyntaxError: ignored

#**GRÁFICOS**

In [ ]:

# GRÁFICO DE GRAND SLAMS POR PILOTO (ORDEM ALFABÉTICA) DADOS SÓ A PARTIR DE 96
grand_slam_count.sort_values(by = 'RACE_NAME', ascending=False).plot.bar(x="DRIVER_NAME", y="RACE_NAME", rot=50, title="Number of Grand Slams per driver (All Time)");

driver_pole.plot.bar(x="DRIVER_NAME", y="GRID_POSITION", rot=50, title="Pole positions per driver (Top 10 All Time)");




In [ ]:
constructor_wins_plot = constructor_wins.copy()
constructor_wins_plot.loc[constructor_wins_plot['RACE_YEAR'] == 2021, '2021_STANDINGS_POINTS'] = constructor_wins_plot['CONSTRUCTOR_STANDINGS_POINTS']
constructor_wins_plot.loc[constructor_wins_plot['RACE_YEAR'] == 2020, '2020_STANDINGS_POINTS'] = constructor_wins_plot['CONSTRUCTOR_STANDINGS_POINTS']
constructor_wins_plot.loc[constructor_wins_plot['RACE_YEAR'] == 2019, '2019_STANDINGS_POINTS'] = constructor_wins_plot['CONSTRUCTOR_STANDINGS_POINTS']
constructor_wins_plot.loc[constructor_wins_plot['RACE_YEAR'] == 2018, '2018_STANDINGS_POINTS'] = constructor_wins_plot['CONSTRUCTOR_STANDINGS_POINTS']
constructor_wins_plot.loc[constructor_wins_plot['RACE_YEAR'] == 2017, '2017_STANDINGS_POINTS'] = constructor_wins_plot['CONSTRUCTOR_STANDINGS_POINTS']

In [ ]:
constructor_wins_plot.loc[constructor_wins_plot['RACE_YEAR'] < 2017]
index_delete = constructor_wins_plot.index[constructor_wins_plot['RACE_YEAR'] < 2017].tolist()
constructor_wins_plot.drop(index_delete,axis=0, inplace = True)
constructor_wins_plot = constructor_wins_plot.set_index('RACE_YEAR')
#constructor_wins_plot.drop(constructor_wins_plot.loc[constructor_wins_plot['RACE_YEAR'] < 2017])

In [ ]:
from matplotlib.pyplot import figure
figure(figsize=(15, 10), dpi=80)

ax = plt.gca()
p1 = plt.bar(constructor_wins['CONSTRUCTOR_NAME'], constructor_wins['CONSTRUCTOR_STANDINGS_POINTS'])
#constructor_wins_plot.plot.bar(x='CONSTRUCTOR_NAME',rot=70,ax=ax, width=2)
plt.show()

In [ ]:
#from matplotlib.pyplot import figure
figure(figsize=(15, 10), dpi=80)

ax = plt.gca()
p1 = plt.bar(constructor_wins_plot['CONSTRUCTOR_NAME'], constructor_wins_plot['2021_STANDINGS_POINTS'])
p2 = plt.bar(constructor_wins_plot['CONSTRUCTOR_NAME'], constructor_wins_plot['2020_STANDINGS_POINTS'])
#constructor_wins_plot.plot.bar(x='CONSTRUCTOR_NAME',rot=70,ax=ax, width=2)
plt.show()

In [ ]:
driver_wins_plot = driver_wins[driver_wins['RACE_YEAR'] == 2021]
driver_wins_plot = driver_wins_plot.set_index(['DRIVER_NAME'])
driver_wins_plot.plot.bar(y='RACE_DRIVER_POINTS', title = '2021 Driver Standings')